In [0]:
# Configuración súper simple con Access Key
storage_account_name = "sistecreditofinal"
storage_account_access_key = "YpYHNOKME38oGXISqD7KFinQ3arvr43JNX59hiWXyTQvj8O7MwMlRQAx/jrPE2bMY+NHAIC0Sub7+AStbzR/Bg=="
container_name = "raw"
spark.conf.set(
    f"fs.azure.account.key.{storage_account_name}.dfs.core.windows.net",
    storage_account_access_key
)

In [0]:
# === CREAR ARCHIVO .env DIRECTAMENTE EN DATABRICKS ===

# Contenido de tu archivo .env
env_content = """AZURE_STORAGE_ACCOUNT=sistecreditofinal
AZURE_STORAGE_KEY=YpYHNOKME38oGXISqD7KFinQ3arvr43JNX59hiWXyTQvj8O7MwMlRQAx/jrPE2bMY+NHAIC0Sub7+AStbzR/Bg=="""

# Crear directorio si no existe
dbutils.fs.mkdirs("/Users/pansezapata@gmail.com/")

# Escribir archivo .env a DBFS
result = dbutils.fs.put("/Users/pansezapata@gmail.com/.env", env_content, overwrite=True)

print(f"Archivo .env creado: {result}")

# Verificar que existe
files = dbutils.fs.ls("/Users/pansezapata@gmail.com/")
print("Archivos en tu directorio:")
for file in files:
    print(f"  - {file.name}")


In [0]:
# === LEER ARCHIVO .env DE FORMA SEGURA ===

%pip install python-dotenv

from dotenv import load_dotenv
import os

# Ruta correcta (sin /dbfs al inicio para load_dotenv)
env_path = "/dbfs/Users/pansezapata@gmail.com/.env"

# Cargar variables desde .env
result = load_dotenv(env_path)
print(f".env cargado: {result}")

# Leer variables de entorno
storage_account = os.getenv('AZURE_STORAGE_ACCOUNT')
storage_key = os.getenv('AZURE_STORAGE_KEY')

print(f"Storage Account: {storage_account}")
print(f"Storage Key: {'*' * 12} (protegida)")

# Verificar que no sean None
if storage_account and storage_key:
    # Configurar Spark
    spark.conf.set(
        f"fs.azure.account.key.{storage_account}.dfs.core.windows.net",
        storage_key
    )
    print("Configuración Spark completada!")
else:
    print("Error: Variables de entorno están vacías")



In [0]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, roc_auc_score, roc_curve
from sklearn.preprocessing import LabelEncoder, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Leer datos (usando el método híbrido que ya funciona)
print("📖 Cargando datos...")
file_path = "abfss://raw@sistecreditofinal.dfs.core.windows.net/data/v1/credit_risk_dataset.csv"
spark_df = spark.read.option("header", "true").option("inferSchema", "true").csv(file_path)
df = spark_df.toPandas()

print(f"Dataset cargado: {df.shape[0]} filas, {df.shape[1]} columnas")
print("Columnas:", list(df.columns))

In [0]:
def explore_credit_data(df):
    """Análisis exploratorio específico para datos de crédito"""
    
    print("🔍 === ANÁLISIS EXPLORATORIO ===")
    print(f"Dimensiones: {df.shape}")
    print(f"Columnas: {list(df.columns)}")
    
    # Información general
    print("\nInformación del dataset:")
    df.info()
    
    # Estadísticas descriptivas
    print("\nEstadísticas descriptivas:")
    print(df.describe())
    
    # Valores nulos
    print("\nValores nulos por columna:")
    nulls = df.isnull().sum()
    if nulls.sum() > 0:
        print(nulls[nulls > 0])
    else:
        print("✅ No hay valores nulos")
    
    # Distribución de la variable objetivo (asumiendo que existe)
    target_cols = [col for col in df.columns if 'default' in col.lower() or 'risk' in col.lower() or 'target' in col.lower()]
    if target_cols:
        target_col = target_cols[0]
        print(f"\nDistribución de la variable objetivo '{target_col}':")
        print(df[target_col].value_counts())
        print("Porcentaje:")
        print(df[target_col].value_counts(normalize=True) * 100)
    
    return target_col if target_cols else None

# Ejecutar EDA
target_column = explore_credit_data(df)